# Importación e Intalación de Librerías

In [4]:
import MySQLdb
import csv
import pandas as pd
import sys
import os

# Ruta de archivos

In [10]:
#Rutas de entrada
path_sd = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_dd = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DOWNLOADED_DATA'))
path_ad = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))

#Rutas salida
path_db = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))
path_dwh = os.path.join(os.getcwd(), "DATAWAREHOUSE")
path_dm_oc = os.path.join(path_dwh,'datamart_oc')
path_dm_oec = os.path.join(path_dwh,'datamart_oec')
path_dm_ci = os.path.join(path_dwh,'datamart_ci')

# Funciones

In [8]:
stopwords = open(os.path.join(path_etl,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]
global puntuacion
puntuacion = '!"#$%&:;\'()*+,-./?@[\]^_`{|}~()'

In [9]:
def normalizar(text):
    text = text.lower() 
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.strip()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords and char!=' ':
            new_text.append(text[i])
    new_text = ' '.join(new_text)
    new_text = new_text.strip()
            
    return new_text

# Extracción de datos

## Catálogo Nacional de Cualificaciones - CNC 

- CNC para Operadores de Capacitación - CNCOC
- CNC para Organismos Evaluadores de  la Conformidad - CNCOEC
- CNC  Perfiles Inhabilitados - CNCPI

### ETL CNCOC

In [11]:
df_cncoc = pd.read_csv(os.path.join(path_sd,'cncoc.csv'))
df_cncoc.insert(4, 'tipo', "oc")
df_cncoc.insert(5, 'estado', "habilitado")

In [12]:
# df_cncoc.info()

### ETL CNCOEC

In [13]:
df_cncoec = pd.read_csv(os.path.join(path_sd,'cncoec0.csv'))
df_cncoec.insert(4, 'tipo', "oec")
df_cncoec.insert(5, 'estado', "habilitado")

In [14]:
# df_cncoec.info()

### ETL CNCPI

In [23]:
df_cncpi = pd.read_csv(os.path.join(path_sd,'cncpi0.csv'))
df_cncpi.insert(1, 'competenecia_laboral', "sin competencia laboral")
df_cncpi.insert(4, 'tipo', "desconocido")
df_cncpi.insert(5, 'estado', "inhabilidato")

In [16]:
# df_cncpi.info()

 ### **CNC** - INTEGRACIÓN DE DATOS

In [24]:
df_cnc = pd.concat([df_cncoc,df_cncoec,df_cncpi],axis=0)
df_cnc = df_cnc.apply(lambda x: x.astype(str).str.lower())

In [26]:
df_cnc = df_cnc.drop_duplicates()

In [29]:
#df_cnc.tail()
#df_cnc.info()

### Guardar datos CNC

In [30]:
df_cnc.to_csv(os.path.join(path_db,'cnc.csv'),index=False)

## Operadores de Capacitación - OC

• [Operadores de Capacitación](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt25) - OC

• [OC - Suspendidos](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt125) - OCS

• [OC - Finalizaron su vigencia de Calificación](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt134) - OCF

• [OC - Cancelados](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacion.jsf#j_idt24:j_idt144) - OCC


### ETL OC

In [48]:
df_oc = pd.read_csv(os.path.join(path_sd,'oc0.csv'))
df_oc['ruc_o_codigo'] = df_oc['ruc_o_codigo'].astype('object')
df_oc["nombre_comercial"].fillna("sin_nombre_comercial", inplace = True) 
df_oc.insert(9, 'estado', df_oc.pop('estado'))

In [49]:
df_oc['ruc_o_codigo'] = df_oc['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [50]:
#df_oc.head()
#df_oc.info()

### ETL OCS

In [51]:
df_ocs = pd.read_csv(os.path.join(path_sd,'ocs.csv'))
df_ocs['ruc_o_codigo'] = df_ocs['ruc_o_codigo'].astype('object')
df_ocs.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_ocs.insert(3, 'telefono', 'Sin Teléfono')
df_ocs.insert(4, 'celular', 'Sin Celular')
df_ocs.insert(5, 'correo_electronico', 'Sin Email')
df_ocs.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_ocs.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_ocs.insert(8, 'canton', 'Sin Cantón')

In [52]:
#df_ocs.head()
#df_ocs.info()

### ETL OCC

In [53]:
df_occ = pd.read_csv(os.path.join(path_sd,'occ.csv'))
df_occ['ruc_o_codigo'] = df_occ['ruc_o_codigo'].astype('str')
df_occ.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_occ.insert(3, 'telefono', 'Sin Teléfono')
df_occ.insert(4, 'celular', 'Sin Celular')
df_occ.insert(5, 'correo_electronico', 'Sin Email')
df_occ.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_occ.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_occ.insert(8, 'canton', 'Sin Cantón')

In [54]:
df_occ['ruc_o_codigo'] = df_occ['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [55]:
#df_occ.head()
#df_occ.info()

### ETL OCF

In [56]:
df_ocf = pd.read_csv(os.path.join(path_sd,'ocf0.csv'))
df_ocf['ruc_o_codigo'] = df_ocf['ruc_o_codigo'].astype('str')
df_ocf.insert(2, 'nombre_comercial', 'Sin Nombre Comercial')
df_ocf.insert(3, 'telefono', 'Sin Teléfono')
df_ocf.insert(4, 'celular', 'Sin Celular')
df_ocf.insert(5, 'correo_electronico', 'Sin Correo Electronico')
df_ocf.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_ocf.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_ocf.insert(8, 'canton', 'Sin Cantón')

In [57]:
df_ocf['ruc_o_codigo'] = df_ocf['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [58]:
#df_ocf.head()
#df_ocf.info()

### DATOS LIMPIOS OC

In [46]:
df_OC = pd.concat([df_oc,df_ocs,df_ocf,df_occ],axis=0)
df_OC = df_OC.apply(lambda x: x.astype(str).str.lower())
df_OC = df_OC.drop_duplicates()
df_OC = df_OC.drop_duplicates(subset='razon_social',keep='first')

NameError: name 'df_oc' is not defined

In [229]:
#df_OC.head()
#df_OC.info()

### Guardar datos OC

In [230]:
df_OC.to_csv(os.path.join(path_etl,'oc.csv'),index=False)

### ETL OC_CC

In [140]:
df_oc_cc = pd.read_csv(os.path.join(path,'oc_cc0.csv'))
df_oc_cc = df_oc_cc.dropna(subset=['area','especialidad','curso','modalidad','carga_horaria'])
df_oc_cc['modalidad'] = df_oc_cc['modalidad'].replace(['VIRTUAL'], 'ONLINE')
df_oc_cc = df_oc_cc.apply(lambda x: x.astype(str).str.lower())
df_oc_cc['carga_horaria'] = df_oc_cc['carga_horaria'].astype('int64')
df_oc_cc = df_oc_cc.drop_duplicates()

In [143]:
#df_oc_cc.head()
#df_oc_cc.info()

### Guardar datos OC_CC

In [262]:
df_oc_cc.to_csv(os.path.join(path_etl,'oc_cc.csv'),index=False)

### ETL OC_CL

In [24]:
df_oc_cl = pd.read_csv(os.path.join(path,'oc_cl0.csv'))
df_oc_cl = df_oc_cl.apply(lambda x: x.astype(str).str.lower())
df_oc_cl = df_oc_cl.drop_duplicates()

In [27]:
#df_oc_cl.head()
#df_oc_cl.info()

### Guardar datos OC_CL

In [29]:
df_oc_cl.to_csv(os.path.join(path_etl,'oc_cl.csv'),index=False)

## Personas Capacitadas por OC - PCOC

• [Búsqueda Personas Capacitadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/personasCapacitadasOperadores.jsf#j_idt24:j_idt25) - PCOC

In [6]:
df_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))
df_pcoc["numero_documento"].fillna("Sin Número Documento", inplace = True)
df_pcoc["apellidos_nombres"].fillna("Sin Apellidos y Nombres", inplace = True)
df_pcoc["numero_horas"].fillna(0, inplace = True)
df_pcoc["numero_horas"]=df_pcoc["numero_horas"].astype('int64')
df_pcoc["razon_social_oc"].fillna("Sin Razón Social OC", inplace = True)
df_pcoc["nombre_comercial_oc"].fillna("Sin Nombre Comercial OC", inplace = True)
df_pcoc = df_pcoc.apply(lambda x: x.astype(str).str.lower())
df_pcoc["numero_horas"]=df_pcoc["numero_horas"].astype('int64')
df_pcoc = df_pcoc.drop_duplicates()    

In [244]:
#df_pcoc.head()
#df_pcoc.info()

In [260]:
df_pcoc.to_csv(os.path.join(path_etl,'pcoc.csv'),index=False)

### Datos Número Documento y Apellidos/Nombre

In [ ]:
df_d_n_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))
df_d_n_pcoc = df_d_n_pcoc[['Numero_Documento','Apellidos_Nombres']]
df_d_n_pcoc['size_doc'] = df_d_n_pcoc['Numero_Documento'].apply(lambda doc: len(doc))
df_d_n_pcoc = df_d_n_pcoc[df_d_n_pcoc['size_doc']==10]
df_d_n_pcoc = df_d_n_pcoc.drop_duplicates(['Numero_Documento'], keep='last')
df_d_n_pcoc.pop('size_doc')
df_d_n_pcoc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181304 entries, 0 to 251987
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Numero_Documento   181304 non-null  object
 1   Apellidos_Nombres  181304 non-null  object
dtypes: object(2)
memory usage: 4.1+ MB


In [ ]:
df_d_n_pcoc.to_csv(path_etl+'doc_name_pcoc.csv',index=False)

## Organismos Evaluadores de  la Conformidad - OEC

• [Organismos Evaluadores de la Conformidad](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt25) - OEC

• [OEC - Suspendidos](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt103) - OECS

• [OEC - Finalizaron su vigencia de Reconocimiento](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt112) - OECF

• [OEC - Cancelados](http://portal.trabajo.gob.ec/setec-portal-web/pages/evaluadoresConformidad.jsf#j_idt24:j_idt122) - OECC

### ETL OEC

In [191]:
df_oec = pd.read_csv(os.path.join(path,'oec0.csv'))
df_oec['celular'] = df_oec['celular'].astype('object')
df_oec = df_oec.apply(lambda x: x.astype(str).str.lower())
df_oec['ruc_o_codigo'] = df_oec['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_oec['celular'] = df_oec['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_oec = df_oec.drop_duplicates()

In [228]:
#df_oec.head()
#df_oec.info()

### ETL OECS

In [205]:
df_oecs = pd.read_csv(os.path.join(path,'oecs.csv'))
df_oecs.insert(2, 'direccion', 'Sin Dirección')
df_oecs.insert(3, 'telefono', 'Sin Teléfono')
df_oecs.insert(4, 'celular', 'Sin Celular')
df_oecs.insert(5, 'correo_electronico', 'Sin Email')
df_oecs.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecs.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecs.insert(8, 'canton', 'Sin Cantón')
df_oecs = df_oecs.apply(lambda x: x.astype(str).str.lower())

In [206]:
df_oecs['ruc_o_codigo'] = df_oecs['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [208]:
#df_oecs.head()
#df_oecs.info()

### ETL OECF

In [222]:
df_oecf = pd.read_csv(os.path.join(path,'oecf0.csv'))
df_oecf.insert(2, 'direccion', 'Sin Dirección')
df_oecf.insert(3, 'telefono', 'Sin Teléfono')
df_oecf.insert(4, 'celular', 'Sin Celular')
df_oecf.insert(5, 'correo_electronico', 'Sin Email')
df_oecf.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecf.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecf.insert(8, 'canton', 'Sin Cantón')
df_oecf = df_oecf.apply(lambda x: x.astype(str).str.lower())
df_oecs['ruc_o_codigo'] = df_oecs['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [212]:
#df_oecf.head()
#df_oecf.info()

### ETL OECC

In [213]:
df_oecc = pd.read_csv(os.path.join(path,'oecc.csv'))
df_oecc.insert(2, 'direccion', 'Sin Dirección')
df_oecc.insert(3, 'telefono', 'Sin Teléfono')
df_oecc.insert(4, 'celular', 'Sin Celular')
df_oecc.insert(5, 'correo_electronico', 'Sin Email')
df_oecc.insert(6, 'numero_resolucion', 'Sin Número Resolución')
df_oecc.insert(7, 'fecha_resolucion', 'Sin Fecha Resolución')
df_oecc.insert(8, 'canton', 'Sin Cantón')                               
df_oecc = df_oecc.apply(lambda x: x.astype(str).str.lower())
df_oecc['ruc_o_codigo'] = df_oecc['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)

In [215]:
#df_oecc.head()
#df_oecc.info()

### DATOS LIMPIOS OEC

In [223]:
df_OEC = pd.concat([df_oec,df_oecs,df_oecf,df_oecc],axis=0)
df_OEC = df_OEC.drop_duplicates()
df_OEC = df_OEC.drop_duplicates(subset='razon_social',keep='first')

In [225]:
#df_OEC.head()
#df_OEC.info()

### Guardar datos OEC

In [259]:
df_OEC.to_csv(os.path.join(path_etl,'oec.csv'),index=False)

### ETL OEC_DR

In [237]:
df_oec_dr = pd.read_csv(os.path.join(path,'oec_dr0.csv'))
df_oec_dr["perfil"].fillna("Sin Perfil", inplace = True)
df_oec_dr["esquema_de_certificacion"].fillna("Sin Esquema de Certificación", inplace = True)
df_oec_dr = df_oec_dr.apply(lambda x: x.astype(str).str.lower())
df_oec_dr['ruc_o_codigo'] = df_oec_dr['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_oec_dr = df_oec_dr.drop_duplicates()

In [240]:
#df_oec_dr.head()
#df_oec_dr.info()

### Guardar datos OEC_DR

In [258]:
df_oec_dr.to_csv(os.path.join(path_etl,'oec_dr.csv'),index=False)

## Personas Certificadas por OEC - PCOEC

• [Búsqueda Personas Certificadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/legitimidadCertificacion.jsf#j_idt24:j_idt25) - PCOEC

In [251]:
df_pcoec = pd.concat([
    pd.read_csv(os.path.join(path,'pcoec0.csv')),
    pd.read_csv(os.path.join(path,'pcoec1.csv')),
    pd.read_csv(os.path.join(path,'pcoec2.csv')),
    pd.read_csv(os.path.join(path,'pcoec3.csv')),
    ],axis=0)  
df_pcoec = df_pcoec.drop(['Ver Certificado'],axis=1)
df_pcoec["Fecha Vigencia"].fillna("Sin Fecha Vigencia", inplace = True)
df_pcoec["Celular"].fillna("Sin Celular", inplace = True)
df_pcoec = df_pcoec.rename(columns={'Número Certificación':'numero_certificacion'})
df_pcoec = df_pcoec.rename(columns={'Fecha Certificación':'fecha_certificacion'})
df_pcoec = df_pcoec.rename(columns={'Fecha Vigencia':'fecha_vigencia'})
df_pcoec = df_pcoec.rename(columns={'Ruc Organismo Evaluador':'ruc_o_codigo'})
df_pcoec = df_pcoec.rename(columns={'Organismo Evaluador':'razon_social'})
df_pcoec = df_pcoec.rename(columns={'Perfil':'perfil'}) 
df_pcoec = df_pcoec.rename(columns={'Nombres':'apellidos_nombres'})
df_pcoec = df_pcoec.rename(columns={'Celular':'celular'})
df_pcoec = df_pcoec.apply(lambda x: x.astype(str).str.lower())
df_pcoec['ruc_o_codigo'] = df_pcoec['ruc_o_codigo'].apply(lambda x: '0'+x if len(x)==12 else x)
df_pcoec['celular'] = df_pcoec['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_pcoec = df_pcoec.drop_duplicates()

In [266]:
#df_pcoec.head()
#df_pcoec.info()

### Guardar datos PCOEC

In [264]:
df_pcoec.to_csv(os.path.join(path_etl,'pcoec.csv'),index=False)

## Capacitadores Independientes - CI

• [Búsqueda de Capacitadores Independientes](http://portal.trabajo.gob.ec/setec-portal-web/pages/operadoresCapacitacionIndependientes.jsf#j_idt24:j_idt25) - CI

### ETL CI

In [7]:
df_ci = pd.read_csv(os.path.join(path,'ci0.csv'))
df_ci = df_ci.apply(lambda x: x.astype(str).str.lower())
df_ci['numero_documento'] = df_ci['numero_documento'].apply(lambda x: '0'+x if len(x)==9 else x)
df_ci['celular'] = df_ci['celular'].apply(lambda x: '0'+x if len(x)==9 else x)
df_ci = df_ci.drop_duplicates()

In [9]:
#df_ci.head()
#df_ci.info()

### Guardar datos CI

In [11]:
df_ci.to_csv(os.path.join(path_etl,'ci.csv'),index=False)

### ETL CI_LCA

In [12]:
df_ci_lca =pd.read_csv(os.path.join(path,'ci_lca0.csv'))
df_ci_lca["carga_horaria"].fillna(0, inplace = True)
df_ci_lca["carga_horaria"]=df_ci_lca["carga_horaria"].astype('int64')
df_ci_lca = df_ci_lca.apply(lambda x: x.astype(str).str.lower())
df_ci_lca["carga_horaria"]=df_ci_lca["carga_horaria"].astype('int64')
df_ci_lca = df_ci_lca.drop_duplicates()

In [289]:
#df_ci_lca.head()
#df_ci_lca.info()

### Guardar datos CI_LCA

In [13]:
df_ci_lca.to_csv(os.path.join(path_etl,'ci_lca.csv'),index=False)

## Personas Capacitadas por CI - PCCI

• [Búsqueda de Personas Capacitadas](http://portal.trabajo.gob.ec/setec-portal-web/pages/personasCapacitadas.jsf#j_idt24:j_idt25) - PCCI

In [14]:
df_pcci =pd.read_csv(os.path.join(path,'pcci.csv'))
df_pcci = df_pcci.apply(lambda x: x.astype(str).str.lower())
df_pcci["numero_horas"]=df_pcci["numero_horas"].astype('int64')
df_pcci = df_pcci.drop_duplicates()

In [295]:
#df_pcci.head()
#df_pcci.info()

### Guardar datos PCCI

In [15]:
df_pcci.to_csv(os.path.join(path_etl,'pcci.csv'),index=False)

### Datos Número Documento y Apellidos/Nombre

In [ ]:
df_d_n_pcci = pd.read_csv(path_etl+'pcci.csv')
df_d_n_pcci = df_d_n_pcci[['Numero_Documento','Apellidos_Nombres']]
df_d_n_pcci['size_doc'] = df_d_n_pcci['Numero_Documento'].apply(lambda doc: len(doc))
df_d_n_pcci = df_d_n_pcci[df_d_n_pcci['size_doc']==10]
df_d_n_pcci = df_d_n_pcci.drop_duplicates(['Numero_Documento'], keep='last')
df_d_n_pcci.pop('size_doc')
df_d_n_pcci.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99000 entries, 0 to 140179
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Numero_Documento   99000 non-null  object
 1   Apellidos_Nombres  99000 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [ ]:
df_d_n_pcci.to_csv(path_etl+'doc_name_pcci.csv',index=False)